In [1]:
import psycopg2
from config import local_host, local_name, local_port, local_user
from creds import local_db_password
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import time

from sim_v1 import simulate

warnings.simplefilter('ignore')
conn = psycopg2.connect(dbname=local_name, user =local_user, host=local_host, password=local_db_password, port =local_port)
cur = conn.cursor()
season = 2023

In [2]:
def check_roll_params(setup=None, param_name='rolling_params'):
    total = 0
    for key in setup[param_name]:
        total += setup[param_name][key]
    print(total)

In [3]:
real_results_sql = f"""

select 
	fixture_id,
	max(team_id) filter (where team_id = team_home) as team_home,
	max(team_id) filter (where team_id = team_away) as team_away,
	max(goals_total) filter (where team_id = team_home) as home_goal,
	max(goals_total) filter (where team_id = team_away) as away_goal,
	max(shots_total) filter (where team_id = team_home) as home_shot,
	max(shots_total) filter (where team_id = team_away) as away_shot,
	max(passes_total) filter (where team_id = team_home) as home_passes,
	max(passes_total) filter (where team_id = team_away) as away_passes,
	max(dribbles_total) filter (where team_id = team_home) as home_dribble,
	max(dribbles_total) filter (where team_id = team_away) as away_dribble
	
from (
select
	fixture_id,
	team_id,
	sum(goals_total::int) as goals_total,
	sum(shots_total::int) as shots_total,
	sum(passes_total::int) as passes_total,
	sum(tackles_total::int) as total_tackles,
	sum(interceptions::int) as interceptions,
	sum(blocks::int) as blocks,
	sum(dribbles_attempted::int) as dribbles_total
from ffl.fixture_player_performance
where fixture_id in (select id from ffl.fixtures where season = '{season}')
group by fixture_id, team_id) a
left join ffl.fixtures as fix on fix.id = fixture_id
group by 1
"""



In [4]:
fixtures_sql = f"""
select id, score_home, score_away from ffl.fixtures where season = '{season}'
"""
fixtures_list = []
cur.execute(fixtures_sql)
for fix in cur.fetchall():
    fixtures_list.append(fix[0])

In [5]:
def confusion_matrix(true_results=None, pred_results=None):
    pred_win_true_win = 0
    pred_win_true_loss = 0
    pred_win_true_tie = 0
    pred_loss_true_win = 0
    pred_loss_true_loss = 0
    pred_loss_true_tie = 0
    pred_tie_true_win = 0
    pred_tie_true_loss = 0
    pred_tie_true_tie = 0
    

    for row in range(len(true_results)):
        if pred_results.iloc[row]['home_goal'] > pred_results.iloc[row]['away_goal'] and true_results.iloc[row]['home_goal'] > true_results.iloc[row]['away_goal']:
            pred_win_true_win += 1
        elif pred_results.iloc[row]['home_goal'] > pred_results.iloc[row]['away_goal'] and true_results.iloc[row]['home_goal'] < true_results.iloc[row]['away_goal']:
            pred_win_true_loss += 1.
        elif pred_results.iloc[row]['home_goal'] > pred_results.iloc[row]['away_goal'] and true_results.iloc[row]['home_goal'] == true_results.iloc[row]['away_goal']:
            pred_win_true_tie += 1
        elif pred_results.iloc[row]['home_goal'] < pred_results.iloc[row]['away_goal'] and true_results.iloc[row]['home_goal'] > true_results.iloc[row]['away_goal']:
            pred_loss_true_win += 1
        elif pred_results.iloc[row]['home_goal'] < pred_results.iloc[row]['away_goal'] and true_results.iloc[row]['home_goal'] < true_results.iloc[row]['away_goal']:
            pred_loss_true_loss += 1
        elif pred_results.iloc[row]['home_goal'] < pred_results.iloc[row]['away_goal'] and true_results.iloc[row]['home_goal'] == true_results.iloc[row]['away_goal']:
            pred_loss_true_tie += 1
        elif pred_results.iloc[row]['home_goal'] == pred_results.iloc[row]['away_goal'] and true_results.iloc[row]['home_goal'] > true_results.iloc[row]['away_goal']:
            pred_tie_true_win += 1
        elif pred_results.iloc[row]['home_goal'] == pred_results.iloc[row]['away_goal'] and true_results.iloc[row]['home_goal'] < true_results.iloc[row]['away_goal']:
            pred_tie_true_loss += 1.
        elif pred_results.iloc[row]['home_goal'] == pred_results.iloc[row]['away_goal'] and true_results.iloc[row]['home_goal'] == true_results.iloc[row]['away_goal']:
            pred_tie_true_tie += 1
        

    confusion_matrix_df = pd.DataFrame({
        'labels':['True Win', 'True Loss', 'True_Tie'],
        'Pred Wins':[pred_win_true_win, pred_win_true_loss, pred_win_true_tie],
        'Pred Loss':[pred_loss_true_win, pred_loss_true_loss, pred_loss_true_tie],
        'Pred Ties':[pred_tie_true_win, pred_tie_true_loss, pred_tie_true_tie]
    })
    print(f'Total Accuracy:   {(pred_win_true_win + pred_loss_true_loss + pred_tie_true_tie) / len(true_results)}')
    print(f'Win Analysis:     Accuracy: {pred_win_true_win/(pred_win_true_win + pred_win_true_loss + pred_win_true_tie)}     ||     Recall: {pred_win_true_win / (pred_win_true_win + pred_loss_true_win + pred_tie_true_win)}')
    print(f'Loss Analysis:     Accuracy: {pred_loss_true_loss/(pred_loss_true_win + pred_loss_true_win + pred_loss_true_tie)}     ||     Recall: {pred_loss_true_loss / (pred_win_true_loss + pred_loss_true_loss + pred_tie_true_loss)}')
    if pred_tie_true_loss + pred_tie_true_loss + pred_tie_true_tie > 0:
        print(f'Tie Analysis:     Accuracy: {pred_tie_true_win/(pred_win_true_tie + pred_loss_true_tie + pred_tie_true_tie)}     ||     Recall: {pred_tie_true_win / (pred_tie_true_win + pred_tie_true_loss + pred_tie_true_tie)}')
    print(confusion_matrix_df.head())
        

In [6]:
simulation_iterations = 200

setup_json = {
    'team_home':None,
    'team_away':None,
    'minutes': 90,
    'ticks_per_minute': 15,
    'rolling_params' : {
        'keep': 0.05,
        'defence_initiation': 0.06,
        'pass': 0.83,
        'shoot':0.015,
        'dribble': 0.045
    },
    'home_advantage':0.005, #plus or minus this percent
    'mod_mode' : True,
    'shooting_accuracy_mod': -0.125, #changes performance by this amount
    'goalie_accuracy_mod': 0.275 #changes performance by this amount
}
check_roll_params(setup_json)

1.0


In [7]:
output_df = pd.DataFrame(columns=['fixture', 'team_home','team_away', 'home_goal', 'away_goal', 'home_shot', 'away_shot', 'home_pass', 'away_pass', 'home_dribble', 'away_dribble', ])
total_time = 0
for fix in fixtures_list:
    # time_start = time.time()
    output_df.loc[len(output_df)] = simulate(fix, setup_json, simulation_iterations)
    # total_time += time.time() - time_start

output_df = output_df.sort_values('fixture', ascending=True)
output_df = output_df.reset_index()
output_df = output_df.drop(['index'], axis=1)
real_results_df = pd.read_sql(real_results_sql, conn)
confusion_matrix(real_results_df, output_df)
# print(f'Fixture list iteration time {total_time}')

Total Accuracy:   0.3868421052631579
Win Analysis:     Accuracy: 0.5064935064935064     ||     Recall: 0.4482758620689655
Loss Analysis:     Accuracy: 0.3360655737704918     ||     Recall: 0.3333333333333333
Tie Analysis:     Accuracy: 0.5542168674698795     ||     Recall: 0.40707964601769914
      labels  Pred Wins  Pred Loss  Pred Ties
0   True Win       78.0         50       46.0
1  True Loss       43.0         41       39.0
2   True_Tie       33.0         22       28.0


In [8]:
output_df.describe()

,fixture,team_home,team_away,home_goal,away_goal,home_shot,away_shot,home_pass,away_pass,home_dribble,away_dribble
count,3.800000e+02,380.000000,380.000000,380.000000,380.000000,380.000000,380.000000,380.000000,380.000000,380.000000,380.000000
mean,1.035320e+06,112.600000,112.600000,1.721053,1.497368,11.070316,10.381250,468.308408,451.279211,16.623395,16.026079
std,1.655571e+02,286.479169,286.479169,0.902111,0.810546,2.645963,2.560039,68.345661,66.725709,2.334311,2.341527
min,1.035037e+06,33.000000,33.000000,0.000000,0.000000,5.340000,3.175000,302.730000,271.755000,9.700000,9.645000
25%,1.035132e+06,39.750000,39.750000,1.000000,1.000000,9.223750,8.533750,419.406250,403.078750,15.093750,14.475000
50%,1.035364e+06,47.500000,47.500000,2.000000,1.000000,10.675000,10.257500,464.962500,446.447500,16.657500,15.952500
75%,1.035458e+06,52.750000,52.750000,2.000000,2.000000,12.935000,12.060000,515.925000,496.401250,18.127500,17.457500
max,1.035553e+06,1359.000000,1359.000000,7.000000,6.000000,18.735000,19.130000,677.300000,639.340000,25.435000,23.020000


In [9]:
real_results_df.describe()

,fixture_id,team_home,team_away,home_goal,away_goal,home_shot,away_shot,home_passes,away_passes,home_dribble,away_dribble
count,3.800000e+02,380.000000,380.000000,380.000000,380.000000,380.000000,380.000000,380.000000,380.000000,380.000000,380.000000
mean,1.035320e+06,112.600000,112.600000,1.728947,1.418421,10.952632,8.852632,485.773684,454.878947,17.239474,16.236842
std,1.655571e+02,286.479169,286.479169,1.326691,1.258712,4.575127,3.993975,151.175591,141.927073,5.825793,5.380676
min,1.035037e+06,33.000000,33.000000,0.000000,0.000000,1.000000,1.000000,189.000000,164.000000,4.000000,2.000000
25%,1.035132e+06,39.750000,39.750000,1.000000,0.000000,8.000000,6.000000,373.750000,349.500000,13.000000,12.000000
50%,1.035364e+06,47.500000,47.500000,2.000000,1.000000,10.000000,8.000000,477.000000,439.000000,17.000000,16.000000
75%,1.035458e+06,52.750000,52.750000,3.000000,2.000000,14.000000,11.000000,592.250000,542.750000,20.250000,20.000000
max,1.035553e+06,1359.000000,1359.000000,6.000000,8.000000,27.000000,22.000000,1000.000000,895.000000,35.000000,32.000000
